In [ ]:
!pip install mediapy

In [ ]:
# Get some utility functions from https://github.com/cvg/Hierarchical-Localization/

%cd /kaggle/working/
!rm -rf /kaggle/working/Hierarchical-Localization
!git clone --quiet --recursive https://github.com/cvg/Hierarchical-Localization/
%cd /kaggle/working/Hierarchical-Localization
!pip install -e .

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d

%cd /kaggle/working/

In [ ]:
import os
import pycolmap
import numpy as np
import mediapy as media
import cv2
from glob import glob
from pathlib import Path
from time import time

In [ ]:
dataset = 'urban'
scene = 'kyiv-puppet-theater'
#src = f'/kaggle/input/image-matching-challenge-2023/train/{dataset}/{scene}'
#src = f'/kaggle/input/minitape3d'
#src=f'/kaggle/input/tape-3d'
src=f'/kaggle/input/shoe-half-size'
#images = [cv2.cvtColor(cv2.imread(im), cv2.COLOR_BGR2RGB) for im in glob(f'/kaggle/input/tape-3d/*')]
images = [cv2.cvtColor(cv2.imread(im), cv2.COLOR_BGR2RGB) for im in glob(f'/kaggle/input/shoe-half-size/*')]

In [ ]:
media.show_images(images, height=300, columns=4)

# DISK

the DISK local feature extractor.

We will have to write code to extract features, and then import them into the colmap database. 

You can find examples here: [imc2023-kornia-starter-pack](https://github.com/ducha-aiki/imc2023-kornia-starter-pack.git).

In [ ]:
%cd /kaggle/working/
!rm -rf /kaggle/working/imc2023-kornia-starter-pack
!git clone --quiet --recursive https://github.com/ducha-aiki/imc2023-kornia-starter-pack.git

import sys
sys.path.append('/kaggle/working/imc2023-kornia-starter-pack')

import kornia as K
import kornia.feature as KF
import torch
import h5py
from fastprogress import progress_bar


In [ ]:
# We will need images in pytorch format
def load_torch_image(fname, device=torch.device('cpu')):
    img = K.image_to_tensor(cv2.imread(fname), False).float() / 255.
    img = K.color.bgr_to_rgb(img.to(device))
    return img

# Load DISK w/o internet connection through Kaggle Models voodoo: while not necessary yet, it will be helpful for offline submissions, which require turning off internet access.
def load_DISK(device=torch.device('cpu')):
    disk = KF.DISK().to(device)
    pretrained_dict = torch.load('/kaggle/input/disk/pytorch/depth-supervision/1/loftr_outdoor.ckpt', map_location=device)
    disk.load_state_dict(pretrained_dict['extractor'])
    disk.eval()
    return disk

In [ ]:
def detect_features(img_fnames,
                    num_feats = 2048,
                    device=torch.device('cpu'),
                    feature_dir = '.featureout'):
    disk = load_DISK(device)
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    
    # We will save features to h5 files, and then will import them into colmap
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in progress_bar(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                timg = load_torch_image(img_path, device=device)
                features = disk(timg, num_feats, pad_if_not_divisible=True)[0]
                kpts, descs = features.keypoints, features.descriptors
            
            # Convert to numpy to store in h5py
            kpts = kpts.reshape(-1, 2).detach().cpu().numpy()
            descs = descs.reshape(-1, 128).detach().cpu().numpy()
            f_kp[key] = kpts
            f_desc[key] = descs
    return

img_fnames =  [fname for fname in glob(f'/kaggle/input/shoe-half-size/*')]
feature_dir = 'disk_features'
device=torch.device('cuda')
detect_features(img_fnames, 5000, device=device, feature_dir=feature_dir)

In [ ]:
%%capture
# kornia_moons for feature visualization
!pip install kornia_moons

In [ ]:
# Let's visualize our detections
from kornia_moons.feature import visualize_LAF

image_index = 10
with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp:
    img1 = load_torch_image(img_fnames[image_index])
    key = img_fnames[image_index].split('/')[-1]
    lafs = KF.laf_from_center_scale_ori(torch.from_numpy(f_kp[key][...]).reshape(1,-1, 2))
    visualize_LAF(img1, lafs)


In [ ]:
# Now we will match our features on GPU with kornia

def match_features(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=15, verbose = True):
    with h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
         h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in progress_bar(index_pairs):
                    idx1, idx2 = pair_idx
                    fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
                    key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
                    desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
                    desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
                    # Matching with mutual nearest neighbor check and Lowe's threshold
                    dists, idxs = KF.match_smnn(desc1, desc2, 0.98)
                    if len(idxs)  == 0:
                        continue
                    n_matches = len(idxs)
                    if verbose:
                        print (f'{key1}-{key2}: {n_matches} matches')
                    group  = f_match.require_group(key1)
                    if n_matches >= min_matches:
                         group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return

# matching all to all
index_pairs = []
for i in range(len(img_fnames)):
    for j in range(i+1, len(img_fnames)):
        index_pairs.append((i,j))

match_features(img_fnames, index_pairs, device=torch.device('cuda'), feature_dir=feature_dir)

In [ ]:
# Import into colmap

from h5_to_db import import_into_colmap
tgt=f'/kaggle/working/'
database_path = f'{tgt}/database_disk.db'
!rm -rf {database_path}
img_dir = src

import_into_colmap(img_dir, database_path=database_path, feature_dir=feature_dir)

In [ ]:
# Now we run colmap. First matching - because it is bundled with RANSAC.
# Don't worry, colmap would not match our features again, it will just estimate geometry.
output_path =  f'{tgt}/disk_reconstruction'
os.makedirs(output_path, exist_ok=True)
pycolmap.match_exhaustive(database_path)

# Then we run reconstruction, as before
maps = pycolmap.incremental_mapping(database_path, img_dir, output_path)

In [ ]:
maps

In [ ]:
# Inspect the largest map generated by DISK + colmap. 
# Note that it might produce more than one reconstruction.

best_index = None
best_num_reg_images = 0
for idx in maps:
    if maps[idx].num_reg_images() > best_num_reg_images:
        best_index = idx
        best_num_reg_images = maps[idx].num_reg_images()

if best_num_reg_images > 0:
    print(f'Looking at reconstruction #{best_index} with {best_num_reg_images} registered images')
    fig = viz_3d.init_figure()
    viz_3d.plot_reconstruction(fig, maps[best_index], color='rgba(0,0,255,0.5)', name="Reconstruction", cs=3, cameras=False)
    viz_3d.plot_reconstruction(fig, maps[best_index], color='rgba(255,0,0,0.5)', name="Reconstruction", cs=3, points=False)
    fig.show()
else:
    print('No reconstruction. :(')